# Azure AI Search con integrazione di NVIDIA NIM e LlamaIndex

In questo notebook, dimostreremo come sfruttare i modelli AI di NVIDIA e LlamaIndex per creare un potente pipeline di Retrieval-Augmented Generation (RAG). Utilizzeremo i LLM e gli embeddings di NVIDIA, li integreremo con Azure AI Search come archivio vettoriale e realizzeremo RAG per migliorare la qualità e l'efficienza della ricerca.

## Vantaggi
- **Scalabilità**: Sfrutta i modelli linguistici di grandi dimensioni di NVIDIA e Azure AI Search per un recupero scalabile ed efficiente.
- **Efficienza dei costi**: Ottimizza la ricerca e il recupero con tecniche di archiviazione vettoriale efficienti e ricerca ibrida.
- **Prestazioni elevate**: Combina potenti LLM con la ricerca vettoriale per risposte più rapide e accurate.
- **Qualità**: Mantieni alta la qualità della ricerca basando le risposte dei LLM su documenti pertinenti recuperati.

## Prerequisiti
- 🐍 Python 3.9 o superiore
- 🔗 [Servizio Azure AI Search](https://learn.microsoft.com/azure/search/)
- 🔗 Chiave API NVIDIA per accedere ai LLM e agli Embeddings di NVIDIA tramite i microservizi NVIDIA NIM

## Funzionalità coperte
- ✅ Integrazione con LLM di NVIDIA (utilizzeremo [Phi-3.5-MOE](https://build.nvidia.com/microsoft/phi-3_5-moe))
- ✅ Embeddings di NVIDIA (utilizzeremo [nv-embedqa-e5-v5](https://build.nvidia.com/nvidia/nv-embedqa-e5-v5))
- ✅ Modalità avanzate di recupero di Azure AI Search
- ✅ Indicizzazione dei documenti con LlamaIndex
- ✅ RAG utilizzando Azure AI Search e LlamaIndex con i LLM di NVIDIA

Iniziamo!


In [ ]:
!pip install azure-search-documents==11.5.1
!pip install --upgrade llama-index
!pip install --upgrade llama-index-core
!pip install --upgrade llama-index-readers-file
!pip install --upgrade llama-index-llms-nvidia
!pip install --upgrade llama-index-embeddings-nvidia
!pip install --upgrade llama-index-postprocessor-nvidia-rerank
!pip install --upgrade llama-index-vector-stores-azureaisearch
!pip install python-dotenv

## Installazione e Requisiti
Crea un ambiente Python utilizzando la versione di Python >3.10.

## Per Iniziare!


Per iniziare, è necessario un `NVIDIA_API_KEY` per utilizzare i modelli NVIDIA AI Foundation:  
1) Crea un account gratuito con [NVIDIA](https://build.nvidia.com/explore/discover).  
2) Clicca sul modello di tua scelta.  
3) Sotto Input, seleziona la scheda Python e clicca su **Get API Key**, quindi su **Generate Key**.  
4) Copia e salva la chiave generata come NVIDIA_API_KEY. Da lì, dovresti avere accesso agli endpoint.  


In [3]:
import getpass
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

if not os.environ.get("NVIDIA_API_KEY", "").startswith("nvapi-"):
    nvidia_api_key = getpass.getpass("Enter your NVIDIA API key: ")
    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"
    os.environ["NVIDIA_API_KEY"] = nvidia_api_key


## Esempio RAG utilizzando LLM e Embedding
### 1) Inizializzare il LLM
`llama-index-llms-nvidia`, noto anche come connettore LLM di NVIDIA, consente di connettersi e generare modelli compatibili disponibili nel catalogo API di NVIDIA. Consulta qui per un elenco di modelli di completamento chat: https://build.nvidia.com/search?term=Text-to-Text

Qui utilizzeremo **mixtral-8x7b-instruct-v0.1**


In [75]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

# Here we are using mixtral-8x7b-instruct-v0.1 model from API Catalog
Settings.llm = NVIDIA(model="microsoft/phi-3.5-moe-instruct", api_key=os.getenv("NVIDIA_API_KEY"))

### 2) Inizializzare l'Embedding
`llama-index-embeddings-nvidia`, noto anche come il connettore di Embeddings di NVIDIA, consente di connettersi e generare da modelli compatibili disponibili nel catalogo API di NVIDIA. Abbiamo selezionato `nvidia/nv-embedqa-e5-v5` come modello di embedding. Consulta qui per un elenco di modelli di embedding di testo: https://build.nvidia.com/nim?filters=usecase%3Ausecase_text_to_embedding%2Cusecase%3Ausecase_image_to_embedding


In [6]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding

Settings.embed_model = NVIDIAEmbedding(model="nvidia/nv-embedqa-e5-v5", api_key=os.getenv("NVIDIA_API_KEY"))

In [76]:
import logging
import sys
import os
import getpass
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.indexes import SearchIndexClient
from IPython.display import Markdown, display
from llama_index.vector_stores.azureaisearch import AzureAISearchVectorStore, IndexManagement


search_service_api_key = os.getenv('AZURE_SEARCH_ADMIN_KEY') or getpass.getpass('Enter your Azure Search API key: ')
search_service_endpoint = os.getenv('AZURE_SEARCH_SERVICE_ENDPOINT') or getpass.getpass('Enter your Azure Search service endpoint: ')
search_service_api_version = "2024-07-01"
credential = AzureKeyCredential(search_service_api_key)

# Index name to use
index_name = "llamaindex-nvidia-azureaisearch-demo"

# Use index client to demonstrate creating an index
index_client = SearchIndexClient(
    endpoint=search_service_endpoint,
    credential=credential,
)

# Use search client to demonstrate using existing index
search_client = SearchClient(
    endpoint=search_service_endpoint,
    index_name=index_name,
    credential=credential,
)

In [ ]:
vector_store = AzureAISearchVectorStore(
    search_or_index_client=index_client,
    index_name=index_name,
    index_management=IndexManagement.CREATE_IF_NOT_EXISTS,
    id_field_key="id",
    chunk_field_key="chunk",
    embedding_field_key="embedding",
    embedding_dimensionality=1024, # dimensionality for nv-embedqa-e5-v5 model
    metadata_string_field_key="metadata",
    doc_id_field_key="doc_id",
    language_analyzer="en.lucene",
    vector_algorithm_type="exhaustiveKnn",
    # compression_type="binary" # Option to use "scalar" or "binary". NOTE: compression is only supported for HNSW
)

In [20]:
from llama_index.core import SimpleDirectoryReader, StorageContext, VectorStoreIndex
from llama_index.core.text_splitter import TokenTextSplitter

# Configure text splitter (nv-embedqa-e5-v5 model has a limit of 512 tokens per input size)
text_splitter = TokenTextSplitter(separator=" ", chunk_size=500, chunk_overlap=10)

# Load documents
documents = SimpleDirectoryReader(
    input_files=["data/txt/state_of_the_union.txt"]
).load_data()
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# Create index with text splitter
index = VectorStoreIndex.from_documents(
    documents,
    transformations=[text_splitter],
    storage_context=storage_context,
)

### 5) Crea un motore di interrogazione per porre domande sui tuoi dati

Ecco una query che utilizza una ricerca vettoriale pura in Azure AI Search e collega la risposta al nostro LLM (Phi-3.5-MOE)


In [69]:
query_engine = index.as_query_engine()
response = query_engine.query("Who did the speaker mention as being present in the chamber?")
display(Markdown(f"{response}"))

 The speaker mentioned the Ukrainian Ambassador to the United States, along with other members of Congress, the Cabinet, and various officials such as the Vice President, the First Lady, and the Second Gentleman, as being present in the chamber.

Ecco una query che utilizza la ricerca ibrida in Azure AI Search.


In [70]:
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.vector_stores.types import VectorStoreQueryMode
from IPython.display import Markdown, display
from llama_index.core.schema import MetadataMode

# Initialize hybrid retriever and query engine
hybrid_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.HYBRID)
hybrid_query_engine = RetrieverQueryEngine(retriever=hybrid_retriever)

# Query execution
query = "What were the exact economic consequences mentioned in relation to Russia's stock market?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))

 The Russian stock market experienced a significant drop, losing 40% of its value. Additionally, trading had to be suspended due to the ongoing situation.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

building a coalition of other freedom-loving nations from Europe and the Americas to Asia and Africa to confront Putin. 

I spent countless hours unifying our European allies. We shared with the world in advance what we knew Putin was planning and precisely how he would try to falsely justify his aggression.  

We countered Russia’s lies with truth.   

And now that he has acted the free world is holding him accountable. 

Along with twenty-seven members of the European Union including France, Germany, Italy, as well as countries like the United Kingdom, Canada, Japan, Korea, Australia, New Zealand, and many others, even Switzerland. 

We are inflicting pain on Russia and supporting the people of Ukraine. Putin is now isolated from the world more than ever. 

Together with our allies –we are right now enforcing powerful economic sanctions. 

We are cutting off Russia’s largest banks from the international financial system.  



#### Analisi della Ricerca Vettoriale
La risposta del LLM cattura accuratamente le principali conseguenze economiche menzionate nel testo di origine riguardo al mercato azionario russo. In particolare, afferma che il mercato azionario russo ha subito un calo significativo, perdendo il 40% del suo valore, e che le negoziazioni sono state sospese a causa della situazione in corso. Questa risposta è in linea con le informazioni fornite nella fonte, indicando che il LLM ha identificato e riassunto correttamente i dettagli rilevanti sull'impatto sul mercato azionario derivante dalle azioni della Russia e dalle sanzioni imposte.

#### Commento sui Nodi di Origine
I nodi di origine forniscono un resoconto dettagliato delle conseguenze economiche che la Russia ha affrontato a causa delle sanzioni internazionali. Il testo evidenzia che il mercato azionario russo ha perso il 40% del suo valore e che le negoziazioni sono state sospese. Inoltre, menziona altre ripercussioni economiche, come la svalutazione del Rublo e l'isolamento più ampio dell'economia russa. La risposta del LLM ha efficacemente sintetizzato i punti critici da questi nodi, concentrandosi sull'impatto sul mercato azionario come richiesto dalla query.


Ora, diamo un'occhiata a una query in cui la Ricerca Ibrida non fornisce una risposta ben fondata:


In [71]:
# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = hybrid_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it does mention that the events discussed are happening in the current era and that the actions taken are in response to Putin's aggression. For the precise date, one would need to refer to external sources or historical records.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

our forces are not engaged and will not engage in conflict with Russian forces in Ukraine.  

Our forces are not going to Europe to fight in Ukraine, but to defend our NATO Allies – in the event that Putin decides to keep moving west.  

For that purpose we’ve mobilized American ground forces, air squadrons, and ship deployments to protect NATO countries including Poland, Romania, Latvia, Lithuania, and Estonia. 

As I have made crystal clear the United States and our Allies will defend every inch of territory of NATO countries with the full force of our collective power.  

And we remain clear-eyed. The Ukrainians are fighting back with pure courage. But the next few days weeks, months, will be hard on them.  

Putin has unleashed violence and chaos.  But while he may make gains on the battlefield – he will pay a continuing high price over the long run. 

And a proud Ukrainian people, who have known 30 years  of independence,

### Ricerca Ibrida: Analisi della Risposta LLM
La risposta del LLM nell'esempio di Ricerca Ibrida indica che il contesto fornito non specifica la data esatta dell'invasione dell'Ucraina da parte della Russia. Questa risposta suggerisce che il LLM sta utilizzando le informazioni disponibili nei documenti di origine, ma riconosce l'assenza di dettagli precisi nel testo.

La risposta è accurata nell'identificare che il contesto menziona eventi legati all'aggressione della Russia, ma non individua la data specifica dell'invasione. Questo dimostra la capacità del LLM di comprendere le informazioni fornite, pur riconoscendo le lacune nel contenuto. Il LLM invita efficacemente l'utente a consultare fonti esterne o registri storici per ottenere la data esatta, mostrando un livello di cautela quando le informazioni sono incomplete.

### Analisi dei Nodi di Origine
I nodi di origine nell'esempio di Ricerca Ibrida contengono estratti di un discorso che discute la risposta degli Stati Uniti alle azioni della Russia in Ucraina. Questi nodi sottolineano l'impatto geopolitico più ampio e le misure adottate dagli Stati Uniti e dai loro alleati in risposta all'invasione, ma non menzionano la data specifica dell'invasione. Questo è coerente con la risposta del LLM, che identifica correttamente che il contesto manca di informazioni precise sulla data.


In [72]:
# Initialize hybrid retriever and query engine
semantic_reranker_retriever = index.as_retriever(vector_store_query_mode=VectorStoreQueryMode.SEMANTIC_HYBRID)
semantic_reranker_query_engine = RetrieverQueryEngine(retriever=semantic_reranker_retriever)

# Query execution
query = "What was the precise date when Russia invaded Ukraine?"
response = semantic_reranker_query_engine.query(query)

# Display the response
display(Markdown(f"{response}"))
print("\n")

# Print the source nodes
print("Source Nodes:")
for node in response.source_nodes:
    print(node.get_content(metadata_mode=MetadataMode.LLM))


 The provided context does not specify the exact date of Russia's invasion of Ukraine. However, it mentions that the event occurred six days before the speech was given. To determine the precise date, one would need to know the date of the speech.



Source Nodes:
file_path: data\txt\state_of_the_union.txt

Madam Speaker, Madam Vice President, our First Lady and Second Gentleman. Members of Congress and the Cabinet. Justices of the Supreme Court. My fellow Americans.  

Last year COVID-19 kept us apart. This year we are finally together again. 

Tonight, we meet as Democrats Republicans and Independents. But most importantly as Americans. 

With a duty to one another to the American people to the Constitution. 

And with an unwavering resolve that freedom will always triumph over tyranny. 

Six days ago, Russia’s Vladimir Putin sought to shake the foundations of the free world thinking he could make it bend to his menacing ways. But he badly miscalculated. 

He thought he could roll into Ukraine and the world would roll over. Instead he met a wall of strength he never imagined. 

He met the Ukrainian people. 

From President Zelenskyy to every Ukrainian, their fearlessness, their courage, their determination, inspires the world. 

### Analisi della risposta LLM con approccio ibrido e reranking
Nell'esempio di approccio ibrido con reranking, la risposta del LLM fornisce un contesto aggiuntivo notando che l'evento è avvenuto sei giorni prima del discorso. Questo indica che il LLM è in grado di dedurre la data dell'invasione basandosi sulla tempistica del discorso, anche se necessita comunque di conoscere la data esatta del discorso per essere preciso.

Questa risposta dimostra una capacità migliorata di utilizzare indizi contestuali per fornire una risposta più informativa. Evidenzia il vantaggio del reranking, dove il LLM può accedere e dare priorità alle informazioni più rilevanti per fornire un'approssimazione più vicina al dettaglio desiderato (ad esempio, la data dell'invasione).

### Analisi dei nodi sorgente
I nodi sorgente in questo esempio includono riferimenti alla tempistica dell'invasione della Russia, menzionando specificamente che è avvenuta sei giorni prima del discorso. Sebbene la data esatta non sia ancora esplicitamente indicata, i nodi forniscono un contesto temporale che consente al LLM di offrire una risposta più sfumata. L'inclusione di questo dettaglio dimostra come il reranking possa migliorare la capacità del LLM di estrarre e dedurre informazioni dal contesto fornito, producendo una risposta più accurata e informativa.


**Nota:**
In questo notebook, abbiamo utilizzato i microservizi NVIDIA NIM dal Catalogo API di NVIDIA.  
Le API sopra menzionate, `NVIDIA (llms)`, `NVIDIAEmbedding`, e [Azure AI Search Semantic Hybrid Retrieval (built-in reranking)](https://learn.microsoft.com/azure/search/semantic-search-overview). Nota che le API sopra citate possono anche supportare microservizi ospitati autonomamente.

**Esempio:**
```python
NVIDIA(model="meta/llama3-8b-instruct", base_url="http://your-nim-host-address:8000/v1")```



---

**Disclaimer**:  
Questo documento è stato tradotto utilizzando il servizio di traduzione automatica [Co-op Translator](https://github.com/Azure/co-op-translator). Sebbene ci impegniamo per garantire l'accuratezza, si prega di notare che le traduzioni automatiche possono contenere errori o imprecisioni. Il documento originale nella sua lingua nativa dovrebbe essere considerato la fonte autorevole. Per informazioni critiche, si raccomanda una traduzione professionale effettuata da un traduttore umano. Non siamo responsabili per eventuali incomprensioni o interpretazioni errate derivanti dall'uso di questa traduzione.
